In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
import pickle

In [2]:
import pandas as pd
df=pd.read_csv(r"C:\Users\Swetha\Downloads\Data Car Resale Value Prediction (3)\Data\autos.csv", header=0, sep=',', encoding='Latin1',)

In [4]:
import pandas as pd
df=pd.read_csv(r"C:\Users\Swetha\Downloads\Data Car Resale Value Prediction (3)\Data\autos.csv", header=0, sep=',', encoding='Latin1',)

print(df.seller.value_counts())
df[df.seller!='gewerblich']
df=df.drop('seller',1)
print(df.offerType.value_counts())
df[df.offerType!='Gesuch']
df=df.drop('offerType',1)

privat        371525
gewerblich         3
Name: seller, dtype: int64


C:\Users\Swetha\AppData\Local\Temp/ipykernel_4504/2840843502.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df=df.drop('seller',1)
C:\Users\Swetha\AppData\Local\Temp/ipykernel_4504/2840843502.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df=df.drop('offerType',1)


Angebot    371516
Gesuch         12
Name: offerType, dtype: int64


In [5]:
print(df.shape)
df=df[(df.powerPS >50) & (df.powerPS <900)]
df=df[(df.yearOfRegistration >=1950) & (df.yearOfRegistration <2017)]
print(df.shape)

(371528, 18)
(309171, 18)


In [6]:
df.drop(['name','abtest','dateCrawled','nrOfPictures','lastSeen','postalCode','dateCreated'],axis='columns',inplace=True)

In [7]:
new_df=df.copy()
new_df=new_df.drop_duplicates(['price','vehicleType','yearOfRegistration','gearbox','powerPS','model','kilometer','monthOfRegistration','fuelType','notRepairedDamage'])

In [8]:
new_df.gearbox.replace(('manuell','automatik'),('manual','automatic'),inplace=True)
new_df.fuelType.replace(('benzin','andere','elektro'),('petrol','others','electric'),inplace=True)
new_df.vehicleType.replace(('kleinwagen','cabrio','kombi','andere'),('small car','convertible','combination','others'),inplace=True)
new_df.notRepairedDamage.replace(('ja','nein'),('Yes','No'),inplace=True)

In [9]:
new_df=new_df[(new_df.price >=100)&(new_df.price <=150000)]
new_df['notRepairedDamage'].fillna(value='not-declared',inplace=True)
new_df['fuelType'].fillna(value='not-declared',inplace=True)
new_df['gearbox'].fillna(value='not-declared',inplace=True)
new_df['vehicleType'].fillna(value='not-declared',inplace=True)
new_df['model'].fillna(value='not-declared',inplace=True)

In [10]:
new_df.to_csv("autos_preprocessed.csv")

In [11]:
from sklearn.preprocessing import LabelEncoder
labels=['gearbox','notRepairedDamage','model','brand','fuelType','vehicleType']
mapper={}
for i in labels:
    mapper[i]=LabelEncoder()
    mapper[i].fit(new_df[i])
    tr=mapper[i].transform(new_df[i])
    np.save(str('classes'+i+'.npy'),mapper[i].classes_)
    print(i,":",mapper[i])
    new_df.loc[:,i+'_labels']=pd.Series(tr,index=new_df.index)
labeled = new_df[['price','yearOfRegistration','powerPS','kilometer','monthOfRegistration']+[x+"_labels" for x in labels]]
print(labeled.columns)

gearbox : LabelEncoder()
notRepairedDamage : LabelEncoder()
model : LabelEncoder()
brand : LabelEncoder()
fuelType : LabelEncoder()
vehicleType : LabelEncoder()
Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


In [12]:
Y=labeled.iloc[:,0].values
X=labeled.iloc[:,1:].values
Y=Y.reshape(-1,1)

In [13]:
from sklearn.model_selection import cross_val_score,train_test_split
X_train,X_test,Y_train,Y_test= train_test_split(X,Y,test_size=0.3,random_state=3)

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
regressor = RandomForestRegressor(n_estimators=1000,max_depth=10,random_state=34)
regressor.fit(X_train,np.ravel(Y_train,order='C'))

RandomForestRegressor(max_depth=10, n_estimators=1000, random_state=34)

In [19]:
y_pred=regressor.predict(X_test)
print(r2_score(Y_test,y_pred))

0.834527626497731


In [20]:
filename='resale_model.sav'
pickle.dump(regressor,open(filename,'wb'))

In [21]:
import pandas as pd
import numpy as np
from flask import Flask,render_template,Response,request
import pickle
from sklearn.preprocessing import LabelEncoder
import pickle

In [22]:
app=Flask(__name__)
filename= 'resale_model.sav'
model_rand = pickle.load(open(filename,'rb'))

In [23]:
@app.route('/')
def index():
    return render_template('resaleintro.html')
@app.route('/predict')
def predict():
    return render_template('resalepredict.html')


In [24]:
@app.route('/y_predict',methods=['GET','POST'])
def y_predict():
    regyear = int(request.form['regyear'])
    powerps =float(request.form['powerps'])
    kms= float(request.form['kms'])
    regmonth = int(request.form.get('regmonth'))
    gearbox = request.form['gearbox']
    damage =request.form['dam']
    model = request.form.get('modeltype')
    brand = request.form.get('brand')
    fuelType = request.form.get('fuel')
    vehicletype = request.form.get('vehicletype')
    new_row = {'yearOfRegistration':regyear,'powerPs':powerps,'kilometer':kms,'monthOfRegistration':regmonth, 'gearbox':gearbox, 'notRepairedDamage':damage, 'model':model, 'brand':brand, 'fuelType':fuelType, 'vehicleType':vehicletype}
    print(new_row)
    new_df = pd.DataFrame(columns =['vehicleType','yearOfRegistration', 'gearbox', 'powerPs', 'model', 'kilometer', 'monthOfRegistration', 'fuleType', 'brand', 'notRepairedDamage'])
    new_df = new_df.append(new_row,ignore_index = True)
    labels = ['gearbox','notRepairedDamage','model','brand','fuelType','vehicleType']
    mapper={}
    for i in labels:
        mapper[i]=LabelEncoder()
        mapper[i].classes_=np.load(str('classes'+i+'.npy'))
        tr=mapper[i].fit_transform(new_df[i])
        new_df.loc[:, i +'labels'] = pd.Series(tr, index = new_df.index)
    labeled = new_df[['yearOfRegistration','powerPs','kilometer','monthOfRegistrstion']+[x+'_labels' for x in labels]]
    x=labeled.values
    print(x)
    y_prediction = model_rand.predict(x)
    print(y_prediction)
    return render_template('resalepredict.html',ypred = 'The resale value predicted is {:.2f}$'.format(y_prediction[0]))

In [26]:
if __name__ == '__main__' :
    app.run(host='localhost',debug=True,threaded=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\Swetha\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
